[![pythonista](img/pythonista.png)](https://www.pythonista.io)

# JSONSchema en Python.

https://json-schema.org/

In [ ]:
!pip install jsonschema 

## Importación de módulos y datos.

In [ ]:
from flask import Flask, jsonify, request, abort
from json import loads, dumps
import jsonschema

In [ ]:
ruta = 'data/alumnos.txt'
carreras = ("Sistemas", "Derecho", "Actuaría", "Arquitectura", "Administración")

In [ ]:
esquema_alumno = {
    "$schema": "http://json-schema.org/draft-07/schema#",
    "type": "object",
    "properties": {
        "Nombre": {"type": "string",
                   "minLength":1,
                   },
        "Primer Apellido": {"type": "string", 
                   "minLength":1,},
        "Segundo Apellido": {"type": "string", 
                   "minLength":1,},
        "Carrera": {"type": "string"},
        "Semestre": {"type": "number",
                   "minimum": 1,
                   "maximum": 50,},
        "Promedio": {"type": "number",
                   "minimum": 0,
                   "maximum": 10,},
        "Al Corriente": {"type": "boolean"},
    },
    "required": ["Nombre",  "Primer Apellido", "Carrera", "Semestre",
                 "Promedio", "Al Corriente"]
}

In [ ]:
jsonschema.validate({'Al Corriente': False,
                     'Carrera': 'Arquitectura',
                     'Nombre': 'Pedro', 
                     'Primer Apellido': 'Solis', 
                     'Promedio': 7.8, 
                     'Semestre': 5,}, 
                    esquema_alumno)

In [ ]:
jsonschema.validate({'Al Corriente': False,
                     'Carrera': 'Arquitectura',
                     'Nombre': 'Pedro', 
                     'Primer Apellido': 'Solis', 
                     'Promedio': 7.8, 
                     'Semestre': 5,
                     'Género': 'F',},
                    esquema_alumno)

In [ ]:
jsonschema.validate({'Al Corriente': False, 
          'Carrera': 'Arquitectura', 
          'Nombre': 'Pedro', 
          'Primer Apellido': 'Solis', 
          'Promedio': 7.8, 
          'Semestre': -7,}, 
                    esquema_alumno)

## Definición de funciones.

### Funciones de gestión de la base de datos.

En este caso la base de datos no es otra cosa más que un archivo de texto que representa a un objeto de tipo *list* de Python. 

La base de datos puede ser consultada en [data/alumnos.txt](data/alumnos.txt). 

### Función de carga de datos.

In [ ]:
def carga_base(ruta):
    with open(ruta, 'tr') as base:
        return eval(base.read())

### Función de escritura de datos.

In [ ]:
def escribe_base(lista ,ruta):
    with open(ruta, 'wt') as base:
            base.write(str(lista))

### Función de búsqueda en la base de datos.
* Busca dentro del campo *'Cuenta'* de cada elemento de *base* al número entero correspondiente al argumento de *cuenta*.
* En caso de encontrar una coincidencia, regresa al elemento.
* En caso de no encontrar coincidencia regresa *False*.

In [ ]:
def busca_base(cuenta, base):
    for alumno in base:
        try:
            if alumno['Cuenta'] == int(cuenta):
                return alumno
        except:
            return False
    return False

## Funciones de validación de datos.

### Función que valida el tipo de dato.

### Función que valida las reglas de los datos.
* Los campos *'Nombre"* y *'Primer Apellido'* no deben de ser una cadena de caracteres vacía.
* El campo 'Semestre' debe de ser un entero mayor a 1.
* La cadena de caracteres del campo 'Carrera' debe de estar dentro de las cadenas listadas en *datos.carrera*.
* El campo promedio debe de ser un número entre 0 y 10.

### Función que valida que el mensaje contiene todos los campos obligatorios.

In [ ]:
def recurso_completo(base, ruta, cuenta, peticion):
    try:
        candidato = {'Cuenta': int(cuenta)}
        peticion = loads(peticion)
        jsonschema.validate(peticion, esquema_alumno)
        if set(esquema_alumno["properties"]).issuperset(peticion) \
        and peticion["Carrera"] in carreras:
            candidato.update(peticion)
        else:
            abort(406)
    except jsonschema.exceptions.ValidationError:
        abort(406)
    # except:
    #    abort(500)
    base.append(candidato)
    escribe_base(base, ruta)
    return jsonify(candidato)

## Código del servidor.

* El servidor correrá en [localhost:5000/api/](localhost:5000/api/). Si se accede a la raíz, se desplegará un listado de todos los alumnos en formato JSON.
* El servidor soporta los métodos:
    * **GET**: para obtener la información de un alumno por su número de cuenta.
    * **POST**: para crear un registro nuevo.
    * **PUT**: para sustituir por completo un registro existente.
    * **PATCH**: para modificar ciertos datos de un registro existente.
    * **DELETE**: para eliminar un registro existente.

In [ ]:
app = Flask(__name__)


@app.route('/api/', methods=['GET'])
def index():
    with open(ruta, 'tr') as base:    
        return jsonify(eval(base.read()))

    
@app.route('/api/<cuenta>', methods=['GET', 'POST', 'PUT', 'PATCH', 'DELETE'])
def api(cuenta):
    
    if request.method == 'GET':
        base = carga_base(ruta)
        alumno = busca_base(cuenta, base)
        if alumno:
            return jsonify(alumno)
        else:
            abort(404)
            
    if request.method == 'DELETE':               
        base = carga_base(ruta)
        alumno = busca_base(cuenta, base)
        if alumno:
            base.remove(alumno)
            escribe_base(base, ruta)
            return jsonify(alumno)
        else:
            abort(404)
        
    if request.method == 'POST':
        base = carga_base(ruta)
        alumno = busca_base(cuenta, base)
        if alumno:
            abort(409)
        else:
            return recurso_completo(base, ruta, cuenta, request.data)
            
    if request.method == 'PUT':
        base = carga_base(ruta)
        alumno = busca_base(cuenta, base)
        if not alumno:
            abort(404)
        else:
            base.remove(alumno)
            return recurso_completo(base, ruta, cuenta, request.data)
        
    if request.method == 'PATCH':
        base = carga_base(ruta)
        alumno = busca_base(cuenta, base)
        if not alumno:
            abort(404)
        else:
            base.remove(alumno)
            alumno.pop('Cuenta')
            alumno.update(loads(request.data))
            print(alumno)
            return recurso_completo(base, ruta, cuenta, dumps(alumno))

In [ ]:
app.run('0.0.0.0')

### Notas:  
* **No reinicie o detenga el kernel de la notebook hasta que los clientes que accedan a esta aplicación hayan terminado sus sesiones.**
* Debido a que el código de la celda de arriba levanta el servidor de Flask, ésta se ejecutará indefinidamente y desplegará los mensajes de respuesta a las peticiones de los clientes que se conecten. 
* La notebook [14_cliente_api_rest.ipynb](12_cliente_api_rest.ipynb) contiene al cliente para este servidor.


<p style="text-align: center"><a rel="license" href="http://creativecommons.org/licenses/by/4.0/"><img alt="Licencia Creative Commons" style="border-width:0" src="https://i.creativecommons.org/l/by/4.0/80x15.png" /></a><br />Esta obra está bajo una <a rel="license" href="http://creativecommons.org/licenses/by/4.0/">Licencia Creative Commons Atribución 4.0 Internacional</a>.</p>
<p style="text-align: center">&copy; José Luis Chiquete Valdivieso. 2021.</p>